Create Delta Tables Using 3 Methods

In [0]:
#Move the file from Workspace to DBFS

dbutils.fs.cp("file:/Workspace/Shared/sales_data.csv", "dbfs:/FileStore/sales_data.csv")



True

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DateType

# Define the schema
customer_schema = StructType([
    StructField("CustomerID", StringType(), True),
    StructField("CustomerName", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("SignupDate", StringType(), True)  # Keep as StringType to convert later
])

# Load the JSON with the defined schema
customer_df = spark.read.format("json").schema(customer_schema).load("/FileStore/customer_data.json")


In [0]:
# Load the CSV into a DataFrame
sales_df = spark.read.format("csv").option("header", "true").load("/FileStore/sales_data.csv")
sales_df.show()

+----------+------+--------+--------+-----+
|      Date|Region| Product|Quantity|Price|
+----------+------+--------+--------+-----+
|2024-09-01| North|Widget A|      10|25.50|
|2024-09-01| South|Widget B|       5|15.75|
|2024-09-02| North|Widget A|      12|25.50|
|2024-09-02|  East|Widget C|       8|22.50|
|2024-09-03|  West|Widget A|      15|25.50|
|2024-09-03| South|Widget B|      20|15.75|
|2024-09-03|  East|Widget C|      10|22.50|
|2024-09-04| North|Widget D|       7|30.00|
|2024-09-04|  West|Widget B|       9|15.75|
+----------+------+--------+--------+-----+



In [0]:
# Write the sales data DataFrame as a Delta Table
sales_df.write.format("delta").mode("overwrite").save("/delta/sales_data")


In [0]:
# Move the file to DBFS
dbutils.fs.cp("file:/Workspace/Shared/customer_data.json", "dbfs:/FileStore/customer_data.json")

# Load the JSON into a DataFrame
customer_df = spark.read.format("json").load("/FileStore/customer_data.json")



In [0]:
# Write the customer data DataFrame as a Delta Table
customer_df.write.format("delta").mode("overwrite").save("/delta/customer_data")

In [0]:
# Create a sample DataFrame
sample_data = [("John", 30), ("Jane", 25), ("Sam", 35)]
columns = ["Name", "Age"]
sample_df = spark.createDataFrame(sample_data, columns)

# Write the DataFrame as a Parquet file
sample_df.write.mode("overwrite").parquet("/FileStore/sample_parquet_file.parquet")


In [0]:
# Load the Parquet file
parquet_df = spark.read.parquet("/FileStore/sample_parquet_file.parquet")

# Convert the Parquet file to a Delta Table
parquet_df.write.format("delta").mode("overwrite").save("/delta/sample_parquet_to_delta")


In [0]:
dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv", "dbfs:/FileStore/new_sales_data.csv")

True

Data Management

In [0]:

# Load new_sales_data.csv into DataFrame
new_sales_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_sales_data.csv")


In [0]:
# Write the new DataFrame as Delta Table
new_sales_df.write.format("delta").mode("overwrite").save("/delta/new_sales_data")

In [0]:
from delta.tables import *

# Load existing Delta table
delta_table =spark.read.format("delta").load("/delta/sales_data")
delta_new_sales=spark.read.format("delta").load("/delta/new_sales_data")
# Create temporary views for SQL operations
delta_table.createOrReplaceTempView("delta_sales_data")
delta_new_sales.createOrReplaceTempView("new_sales_data")

In [0]:
# Merge new sales data into existing Delta Table

spark.sql("""
          MERGE INTO delta_sales_data AS target
          USING new_sales_data AS source
          ON target.OrderID = source.OrderID
          WHEN MATCHED THEN UPDATE SET 
              target.Date = source.OrderDate, 
              target.CustomerID = source.CustomerID,   
              target.Product = source.Product,
              target.Quantity = source.Quantity, 
              target.Price = source.Price
          WHEN NOT MATCHED THEN INSERT (OrderID, OrderDate, Customer_ID, Product, Quantity, Price)
          VALUES (source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price)
""")

Optimize Delta Table

In [0]:
# Optimize the Delta Table and Z-Order by CustomerID
spark.sql("""
	OPTIMIZE delta_sales1_table ZORDER BY CustomerID
""")

Advanced Features

In [0]:
# Describe the history of the Delta Table
spark.sql("DESCRIBE HISTORY delta.`/delta/sales_data`").show(truncate=False)

+-------+-------------------+----------------+----------------------------------+------------+-------------------------------------------------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+------------------------------------------+
|version|timestamp          |userId          |userName                          |operation   |operationParameters                                                            |job |notebook          |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                                                                                                                                             

In [0]:
# Perform VACUUM to remove old files from the Delta Table
spark.sql("VACUUM delta.`/delta/sales_data`")

DataFrame[path: string]

Hands-on Exercises

In [0]:
# Query an older version of the Delta Table using Time Travel
sales_df_version_1 = spark.read.format("delta").option("versionAsOf", 1).load("/delta/sales_data")
sales_df_version_1.show()


+----------+------+--------+--------+-----+
|      Date|Region| Product|Quantity|Price|
+----------+------+--------+--------+-----+
|2024-09-01| North|Widget A|      10|25.50|
|2024-09-01| South|Widget B|       5|15.75|
|2024-09-02| North|Widget A|      12|25.50|
|2024-09-02|  East|Widget C|       8|22.50|
|2024-09-03|  West|Widget A|      15|25.50|
|2024-09-03| South|Widget B|      20|15.75|
|2024-09-03|  East|Widget C|      10|22.50|
|2024-09-04| North|Widget D|       7|30.00|
|2024-09-04|  West|Widget B|       9|15.75|
+----------+------+--------+--------+-----+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Sample data
data = [("John Doe", 30), ("Jane Doe", 25)]

# Define schema
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True)
])

# Create DataFrame
sales_df = spark.createDataFrame(data, schema)



In [0]:
# Enforce schema when writing to a Delta Table
sales_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/delta/sales_data")

In [0]:
# Optimize the Delta Table
spark.sql("OPTIMIZE delta.`/delta/sales_data`")

# Perform VACUUM
spark.sql("VACUUM delta.`/delta/sales_data`")

DataFrame[path: string]